In [21]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
import krwordrank
from krwordrank.word import KRWordRank
  
from collections import defaultdict
from gensim import models
import fasttext
import fasttext.util
from gensim.models import KeyedVectors

In [10]:
df = pd.read_csv('./data/review_final2.csv')
df

,Unnamed: 0,식당이름,주소,리뷰
0,0,육회자매집,서울특별시 종로구 종로4가 177,토요일 저녁 피크 시간에 갔는데 15분 정도 기다리고 먹은 것 같아요 3호점은 안 ...
1,1,육회자매집,서울특별시 종로구 종로4가 177,1호점은 화장실이 없고 화장실은 2호점에 있음. 음식이 빨리 제공되며 맛은 일품.가...
2,2,육회자매집,서울특별시 종로구 종로4가 177,광장시장내 육회집이구요생각보다 사람정말많아요육회는정말맛있어요 강추입니다
3,3,육회자매집,서울특별시 종로구 종로4가 177,-아마 서울에서 육회로 가장 유명한 곳 중 하나가 아닐까?- 오랜만에 광장시장에 방...
4,4,육회자매집,서울특별시 종로구 종로4가 177,몇년전에 줄서서 먹었던 기억이 있어 다시 갔는데 그 때보단 맛있게 먹진 않았어요. ...
...,...,...,...,...
344682,12558,원스인어블루문,NaN,이것은 뉴욕 스타일 재즈 클럽도 뉴욕 가격. 좋은 단단한 음식과 좋은 와인과 재...
344683,12559,원스인어블루문,NaN,우리는 기회를 여러 라이브 공연 에서는 재즈 바 년 중 최고였어요. 로컬 음식도 ...
344684,12560,원스인어블루문,NaN,여기에 가서 데이트 나의 약혼녀. 은 위대한 재즈 음악 맛있는 이탈리아 음식 ...
344685,12561,원스인어블루문,NaN,블루 재즈 바 / 레스토랑 에 잘 모르 는 문 남쪽 의 Aphujeong 로데오 ...


In [13]:
# 리뷰 데이터가 없는 행 삭제하기
df = df.dropna(subset=['리뷰'])
df

,Unnamed: 0,식당이름,주소,리뷰
0,0,육회자매집,서울특별시 종로구 종로4가 177,토요일 저녁 피크 시간에 갔는데 15분 정도 기다리고 먹은 것 같아요 3호점은 안 ...
1,1,육회자매집,서울특별시 종로구 종로4가 177,1호점은 화장실이 없고 화장실은 2호점에 있음. 음식이 빨리 제공되며 맛은 일품.가...
2,2,육회자매집,서울특별시 종로구 종로4가 177,광장시장내 육회집이구요생각보다 사람정말많아요육회는정말맛있어요 강추입니다
3,3,육회자매집,서울특별시 종로구 종로4가 177,-아마 서울에서 육회로 가장 유명한 곳 중 하나가 아닐까?- 오랜만에 광장시장에 방...
4,4,육회자매집,서울특별시 종로구 종로4가 177,몇년전에 줄서서 먹었던 기억이 있어 다시 갔는데 그 때보단 맛있게 먹진 않았어요. ...
...,...,...,...,...
344682,12558,원스인어블루문,NaN,이것은 뉴욕 스타일 재즈 클럽도 뉴욕 가격. 좋은 단단한 음식과 좋은 와인과 재...
344683,12559,원스인어블루문,NaN,우리는 기회를 여러 라이브 공연 에서는 재즈 바 년 중 최고였어요. 로컬 음식도 ...
344684,12560,원스인어블루문,NaN,여기에 가서 데이트 나의 약혼녀. 은 위대한 재즈 음악 맛있는 이탈리아 음식 ...
344685,12561,원스인어블루문,NaN,블루 재즈 바 / 레스토랑 에 잘 모르 는 문 남쪽 의 Aphujeong 로데오 ...


In [14]:
def get_hangul_from_df(df, col = 'Review'):
    df[col] = df[col].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    return df

In [16]:
df = get_hangul_from_df(df, col ='리뷰')
df

c:\users\kem\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
c:\users\kem\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,식당이름,주소,리뷰
0,0,육회자매집,서울특별시 종로구 종로4가 177,토요일 저녁 피크 시간에 갔는데 분 정도 기다리고 먹은 것 같아요 호점은 안 기다리...
1,1,육회자매집,서울특별시 종로구 종로4가 177,호점은 화장실이 없고 화장실은 호점에 있음 음식이 빨리 제공되며 맛은 일품가격은 대...
2,2,육회자매집,서울특별시 종로구 종로4가 177,광장시장내 육회집이구요생각보다 사람정말많아요육회는정말맛있어요 강추입니다
3,3,육회자매집,서울특별시 종로구 종로4가 177,아마 서울에서 육회로 가장 유명한 곳 중 하나가 아닐까 오랜만에 광장시장에 방문해서...
4,4,육회자매집,서울특별시 종로구 종로4가 177,몇년전에 줄서서 먹었던 기억이 있어 다시 갔는데 그 때보단 맛있게 먹진 않았어요 그...
...,...,...,...,...
344682,12558,원스인어블루문,NaN,이것은 뉴욕 스타일 재즈 클럽도 뉴욕 가격 좋은 단단한 음식과 좋은 와인과 재즈...
344683,12559,원스인어블루문,NaN,우리는 기회를 여러 라이브 공연 에서는 재즈 바 년 중 최고였어요 로컬 음식도 좋...
344684,12560,원스인어블루문,NaN,여기에 가서 데이트 나의 약혼녀 은 위대한 재즈 음악 맛있는 이탈리아 음식 ...
344685,12561,원스인어블루문,NaN,블루 재즈 바 레스토랑 에 잘 모르 는 문 남쪽 의 로데오 역 음식 도 괜찮...


In [17]:
restaurants = df['식당이름'].unique()

In [18]:
def get_wordrank_keywords(texts, beta=0.85, max_iter=10):
    wordrank_extractor = KRWordRank(
        min_count = 4, # 단어의 최소 출현 빈도수 (그래프 생성 시)
        max_length = 10, # 단어의 최대 길이
        verbose = True
    )
    keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter, num_keywords=100)
    return keywords, rank, graph

# 리뷰에서 키워드들을 등장횟수와 함께 추출합니다. cols = [restaurant, review]
def get_keywords_counter(restaurants, train_data, cols):
    counter = defaultdict(lambda: 0)
    for restaurant in restaurants:
        try:
            res_data = train_data[train_data[cols[0]] == restaurant]
            texts = get_texts(res_data, cols[1])
            keywords, rank, graph = get_wordrank_keywords(texts)
            for keyword in keywords.keys():
                counter[keyword] += 1    
        except :
            pass

    res = sorted(counter.items(), key=(lambda x: x[1]), reverse=True)
    return res

In [22]:
keywords_counter = get_keywords_counter(restaurants, df, ['식당이름', '리뷰'])

In [23]:
print(keywords_counter)

[]


In [25]:
# This Python file uses the following encoding: utf-8
import os, sys

from collections import defaultdict
import pickle

import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec

import krwordrank
from krwordrank.word import KRWordRank
from krwordrank.word import summarize_with_keywords


# 사용하실 때, data 변수에 사용하고자 하는 csv 파일을 첨부해서 사용해주시면 감사하겠습니다. 

def get_hangul_from_df(df, col = 'Review'):
    df[col] = df[col].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    return df

def get_texts_scores(fname):
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f]
        docs = [doc for doc in docs if len(doc) == 2]
        
        if not docs:
            return [], []
        
        texts, scores = zip(*docs)
        return list(texts), list(scores)

# negative <= base < positive (neg = 0, pos = 1) 
def devide_rates(df, col = 'Total', base = 3):
    df[col] = df[col].apply(lambda x: 0 if x <= base else 1)
    return df

def get_unique_items(df, col = 'Restaurant'):
    return df[col].unique() 

def get_texts(df, col = 'Review'):
    texts = []
    if df.empty:
        return []
    texts = [review for review in df[col]]
    return texts

def get_pos_neg_texts(df):
    pos, neg = [], []
    if df.empty:
        return [], []
    pos = [review for review, total in zip(df['Review'], df['Total']) if total == 1]
    neg = [review for review, total in zip(df['Review'], df['Total']) if total == 0]

    return pos, neg

# beta : PageRank의 decaying factor beta
def get_wordrank_keywords(texts, beta=0.85, max_iter=10):
    wordrank_extractor = KRWordRank(
        min_count = 3, # 단어의 최소 출현 빈도수 (그래프 생성 시)
        max_length = 10, # 단어의 최대 길이
        verbose = True
    )
    keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter, num_keywords=30)
    return keywords, rank, graph

# 리뷰에서 키워드들을 등장횟수와 함께 추출합니다. cols = [restaurant, review]
def get_keywords_counter(restaurants, train_data, cols):
    counter = defaultdict(lambda: 0)
    for restaurant in restaurants:
        try:
            res_data = train_data[train_data[cols[0]] == restaurant]
            texts = get_texts(res_data, cols[1])
            keywords, rank, graph = get_wordrank_keywords(texts)
            for keyword in keywords.keys():
                counter[keyword] += 1    
        except :
            pass

    res = sorted(counter.items(), key=(lambda x: x[1]), reverse=True)
    return res

# 리뷰에서 키워드들을 추출하고 레스토랑 이름과 함께 묶어 dataframe으로 반환합니다. cols = [restaurant, review]
def get_keywords_each(restaurants, train_data, cols, stopwords = {}):
    #데이터 프레임 구조 설정 
    df = pd.DataFrame(columns=['Restaurant', 'Keywords'])
    for restaurant in restaurants:
        try:
            res_data = train_data[train_data[cols[0]] == restaurant]
            texts = get_texts(res_data, cols[1])
            keywords, rank, graph = get_wordrank_keywords(texts)
            if len(keywords) == 0:
                continue
            else:
                keyword_list = []
                for keyword in keywords.keys():
                    if keyword not in stopwords:
                        keyword_list.append(keyword)
                if len(keyword_list) != 0:
                    df.loc[len(df)] = { 
                        'Restaurant':restaurant,
                        'Keywords':keyword_list
                    }
        except :
            pass

    return df

def summarize_keywords(restaurants, train_data, cols, stopwords):
    df = pd.DataFrame(columns=['Restaurant', 'Keywords'])
    for restaurant in restaurants:
        try:
            res_data = train_data[train_data[cols[0]] == restaurant]
            texts = get_texts(res_data, cols[1])
            keywords = summarize_with_keywords(texts, num_keywords=20, min_count=3,\
                max_length=10, beta=0.85, max_iter=10, stopwords=stopwords, verbose=True)
            if len(keywords) == 0:
                continue
            else:
                keyword_list = []
                for keyword in keywords.keys():
                    if keyword not in stopwords:
                        keyword_list.append(keyword)
                if len(keyword_list) != 0:
                    df.loc[len(df)] = { 
                        'Restaurant':restaurant,
                        'Keywords':keyword_list
                    }
        except :
            pass

    return df

In [26]:
data = df
data

,Unnamed: 0,식당이름,주소,리뷰
0,0,육회자매집,서울특별시 종로구 종로4가 177,토요일 저녁 피크 시간에 갔는데 분 정도 기다리고 먹은 것 같아요 호점은 안 기다리...
1,1,육회자매집,서울특별시 종로구 종로4가 177,호점은 화장실이 없고 화장실은 호점에 있음 음식이 빨리 제공되며 맛은 일품가격은 대...
2,2,육회자매집,서울특별시 종로구 종로4가 177,광장시장내 육회집이구요생각보다 사람정말많아요육회는정말맛있어요 강추입니다
3,3,육회자매집,서울특별시 종로구 종로4가 177,아마 서울에서 육회로 가장 유명한 곳 중 하나가 아닐까 오랜만에 광장시장에 방문해서...
4,4,육회자매집,서울특별시 종로구 종로4가 177,몇년전에 줄서서 먹었던 기억이 있어 다시 갔는데 그 때보단 맛있게 먹진 않았어요 그...
...,...,...,...,...
344682,12558,원스인어블루문,NaN,이것은 뉴욕 스타일 재즈 클럽도 뉴욕 가격 좋은 단단한 음식과 좋은 와인과 재즈...
344683,12559,원스인어블루문,NaN,우리는 기회를 여러 라이브 공연 에서는 재즈 바 년 중 최고였어요 로컬 음식도 좋...
344684,12560,원스인어블루문,NaN,여기에 가서 데이트 나의 약혼녀 은 위대한 재즈 음악 맛있는 이탈리아 음식 ...
344685,12561,원스인어블루문,NaN,블루 재즈 바 레스토랑 에 잘 모르 는 문 남쪽 의 로데오 역 음식 도 괜찮...


In [27]:
restaurants

array(['육회자매집', '순희네 빈대떡', '부촌육회', ..., '심야식당 시즌2 주바리 프로젝트',
       '브릭 오븐 뉴욕 피자', '원스인어블루문'], dtype=object)

In [28]:
stopwords = {'먹었습니다', '주셔서', '맛있', '너무', '맛이', '정말', '있는', '좋아', '괜찮', '음식', '많이', '진짜', '생각', '느낌', '먹었', '메뉴', '여기', '좋았', '그냥', '먹어', '주문', '먹고', '아니', '먹을', '다른', '조금', '좋은', '들어', '맛은', '정도', '엄청', '있어', '방문', '먹는', '사람', '아주', '하나', '같이', 'ㅎㅎ', '있다', '가게', '별로', '양이', '먹으', '그런', '처음', '같은', '맛도', '그리고', '약간', 'ㅋㅋ', '않고', '있었', '같아요', '좋고', '같아', '살짝', '그래도', '역시', '제일', 'ㅠㅠ', '곳이', '하는', '좋아요', '기대', '나오는', '먹기', '다시', '많아', '식당', '근데', '완전', '하지만', '양도', '시간', '항상', '만족', '시켜', '자주', '시켰는데', '있어서', '같다', '나오', '매우', '자리', '추가', '특히', '맛을', '가장', '식사', '테이블', '사장님', '많고', '많은', '없는', '함께', '나는', '생각보다', '적당', '모두', '친구', '배달', '바로', '좋다', '하고',  '굉장히', '무난', '이런', '저는', '종류', '않은', '요리', '먹었는데', '좋았어요', '아닌', '계속', '기본', '많아서', '그래', '아쉬', '부드', '해서', '편이', '손님', '재료', '근처', '않았', '이렇게', '않아', '한번', '리뷰', '뭔가', '개인적으로', '느껴', '시켰', '직원', '나온', '다음', '있고', '먹으면', '가서', '제가', '비해', '포장', '구워', '잘먹었습니다', '거의', '갔는데', '인분', '빠르고', '그리', '빨리', '내가', '안에', '다만', '공간', '직접', '기분', '그렇', '감사합니다', '먹은', '그런지', '간이', '마시', '기다', '사람이', '적당히', '사실', 'ㅋㅋㅋ', '매장', '동네', '들어가', '위에', '향이', '했는데', '느낌이', '없어', '따로', '일단', '시키', '원래', '없이', '분위', '오늘', '빵이', '친절하', '평범', '면이', '다음에', '대기', '분들', '것이', '맛잇', '가격이', 'ㅜㅜ', '좋네요',  '보니', '위치', '깔끔하고', '오랜만에', '요즘', '가면', '좋을', '때문에', '찾아', '맛있었', '나쁘지', '오래', '들어간', '양은', '맛있어',  '모르겠', '것도',  '맛나', '보통', '냄새' '이곳' '있습니다', '선택', '작은', '가는', '내부', '있음', '그렇게', '모르', '않는', '괜찮은', '사이', '달지', '그래서', '이스크림', '좋아하', '좋습니다', '오픈', '와서', '개인', '집이', '없었', '나왔', '없다', '일반', '좋아하는', '훨씬', '주신', '웨이팅이', '서비', '기억', '밥이', '자체', '넣어', '감사', '여러', '전체적으로', '분위기도', '시킬', '좋았다', '나와', '가득', '구성', '예쁘', '가격도', '우리', '올라', '싶은', '아쉬웠', '이름', '어울', '때문', '하지', '없고', '이상', '상당히', '없어서', '않아서', '조합', '언제나', '합니다',  '아니라', '굳이', '식감이', '한다', '이집', '번창하세요', '단계', '있지만',  '아쉽', '좋았습니다', '모든', '찍어', '가끔', '호점', '워낙', '나름', '있을', '배달도', '부분', '여긴', '중에', '진하고'}
keywords_df = summarize_keywords(restaurants, data, ['식당이름', '리뷰'], stopwords)

print(keywords_df)


scan vocabs ... 
num vocabs = 4105
done = 10
scan vocabs ... 
num vocabs = 1102
done = 10
scan vocabs ... 
num vocabs = 446
done = 10
scan vocabs ... 
num vocabs = 251
done = 10
scan vocabs ... 
num vocabs = 148
done = 10
scan vocabs ... 
num vocabs = 35
done = 10
scan vocabs ... 
num vocabs = 121
done = 10
scan vocabs ... 
num vocabs = 64
done = 10
scan vocabs ... 
num vocabs = 83
done = 10
scan vocabs ... 
num vocabs = 69
done = 10
scan vocabs ... 
num vocabs = 15
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 57
done = 10
scan vocabs ... 
num vocabs = 73
done = 10
scan vocabs ... 
num vocabs = 25
done = 10
scan vocabs ... 
num vocabs = 34
done = 10
scan vocabs ... 
num vocabs = 54
done = 10
scan vocabs ... 
num vocabs = 722
done = 10
scan vocabs ... 
num vocabs = 31
done = 10
scan vocabs ... 
num vocabs = 15
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 169
done = 10
scan vocabs ... 
num vocabs = 17
done = 7 Early stopped.
scan vocabs ... 
num vocabs = 37
done = 10
sc

done = 8 Early stopped.
scan vocabs ... 
num vocabs = 8
done = 10
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 9
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 9
done = 10
scan vocabs ... 
num vocabs = 217
done = 10
scan vocabs ... 
num vocabs = 45
done = 10
scan vocabs ... 
num vocabs = 80
done = 10
scan vocabs ... 
num vocabs = 8
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 309
done = 10
scan vocabs ... 
num vocabs = 20
done = 10
scan vocabs ... 
num vocabs = 10
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 2
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 13
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 19
done = 10
scan vocabs ... 
num vocabs = 

done = 1 Early stopped.
scan vocabs ... 
num vocabs = 27
done = 10
scan vocabs ... 
num vocabs = 83
done = 10
scan vocabs ... 
num vocabs = 2
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 14
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 8
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 2
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 4
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 9
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 78
done = 10
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 43
done = 10
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 2
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 8
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 16
done = 10
scan vocabs ... 
num vocabs = 5
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 0


done = 10
scan vocabs ... 
num vocabs = 1625
done = 10
scan vocabs ... 
num vocabs = 13
done = 8 Early stopped.
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 25
done = 10
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 8
done = 10
scan vocabs ... 
num vocabs = 21
done = 10
scan vocabs ... 
num vocabs = 129
done = 10
scan vocabs ... 
num vocabs = 22
done = 10
scan vocabs ... 
num vocabs = 41
done = 10
scan vocabs ... 
num vocabs = 118
done = 10
scan vocabs ... 
num vocabs = 17
done = 10
scan vocabs ... 
num vocabs = 17
done = 10
scan vocabs ... 
num vocabs = 16
done = 10
scan vocabs ... 
num vocabs = 16
done = 10
scan vocabs ... 
num vocabs = 95
done = 10
scan vocabs ... 
num vocabs = 66
done = 10
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 614
done = 10
scan vocabs ... 
num vocabs = 374
done = 10
scan vocabs ... 
num vocabs = 354
done = 10
scan vocabs ... 
num vocabs = 674
done = 10
scan vocabs ... 
num vocabs = 62

done = 10
scan vocabs ... 
num vocabs = 22
done = 10
scan vocabs ... 
num vocabs = 25
done = 10
scan vocabs ... 
num vocabs = 24
done = 10
scan vocabs ... 
num vocabs = 19
done = 10
scan vocabs ... 
num vocabs = 36
done = 10
scan vocabs ... 
num vocabs = 28
done = 10
scan vocabs ... 
num vocabs = 25
done = 10
scan vocabs ... 
num vocabs = 58
done = 10
scan vocabs ... 
num vocabs = 129
done = 10
scan vocabs ... 
num vocabs = 59
done = 10
scan vocabs ... 
num vocabs = 62
done = 10
scan vocabs ... 
num vocabs = 39
done = 10
scan vocabs ... 
num vocabs = 16
done = 10
scan vocabs ... 
num vocabs = 34
done = 10
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 17
done = 10
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 8
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 8
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 58
done = 10
scan vocabs ... 
num vocabs = 18
done = 10
scan vocabs ... 
num vocabs = 1829
done = 10
scan vocabs ... 
num vocabs = 180
don

...

done = 10
scan vocabs ... 
num vocabs = 23
done = 10
scan vocabs ... 
num vocabs = 14
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 9
done = 10
scan vocabs ... 
num vocabs = 186
done = 10
scan vocabs ... 
num vocabs = 8
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 2
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 242
done = 10
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 89
done = 10
scan vocabs ... 
num vocabs = 17
done = 10
scan vocabs ... 
num vocabs = 14
done = 10
scan vocabs ... 
num vocabs = 5
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 7
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 19
done = 10
scan vocabs ... 
num vocabs = 26
done = 10
scan vocabs ... 
num vocabs = 3
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 7
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 15
done = 10
scan vocabs ... 
num vocabs = 9
done = 10
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs 

done = 10
scan vocabs ... 
num vocabs = 4
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 74
done = 10
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 4
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 11
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 16
done = 10
scan vocabs ... 
num vocabs = 16
done = 10
scan vocabs ... 
num vocabs = 16
done = 10
scan vocabs ... 
num vocabs = 11
done = 10
scan vocabs ... 
num vocabs = 2
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 2
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 26
done = 10
scan vocabs ... 
num vocabs = 3
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 16
done = 10
scan vocabs ... 
num vocabs = 5
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 8
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 11
done = 10
scan vocabs ... 
num vocabs = 10
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 5
done = 1 Early stopped.
scan vocabs ... 
num vocabs 

done = 10
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 26
done = 10
scan vocabs ... 
num vocabs = 7
done = 10
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 4
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 14
done = 10
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 5
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 20
done = 10
scan vocabs ... 
num vocabs = 4
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 20
done = 10
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 5
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 4
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 7
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 12
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vo

done = 10
scan vocabs ... 
num vocabs = 14
done = 10
scan vocabs ... 
num vocabs = 22
done = 10
scan vocabs ... 
num vocabs = 48
done = 10
scan vocabs ... 
num vocabs = 204
done = 10
scan vocabs ... 
num vocabs = 107
done = 10
scan vocabs ... 
num vocabs = 10
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 176
done = 10
scan vocabs ... 
num vocabs = 23
done = 10
scan vocabs ... 
num vocabs = 80
done = 10
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 15
done = 10
scan vocabs ... 
num vocabs = 1
scan vocabs ... 
num vocabs = 28
done = 10
scan vocabs ... 
num vocabs = 138
done = 10
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 11
done = 10
scan vocabs ... 
num vocabs = 21
done = 10
scan vocabs ... 
num vocabs = 3
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 0
scan vocabs ... 
num vocabs = 87
done = 10
scan vocabs ... 
num vocabs = 7
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 1

scan vocabs ... 
num vocabs = 8
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 8
done = 10
scan vocabs ... 
num vocabs = 19
done = 10
scan vocabs ... 
num vocabs = 24
done = 10
scan vocabs ... 
num vocabs = 6
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 12
done = 7 Early stopped.
scan vocabs ... 
num vocabs = 58
done = 10
scan vocabs ... 
num vocabs = 9
done = 1 Early stopped.
scan vocabs ... 
num vocabs = 14
done = 10
scan vocabs ... 
num vocabs = 15
done = 10 Early stopped.
scan vocabs ... 
num vocabs = 10
done = 10
scan vocabs ... 
num vocabs = 637
done = 10
scan vocabs ... 
num vocabs = 3742
done = 10 Early stopped.
scan vocabs ... 
num vocabs = 574
done = 10
scan vocabs ... 
num vocabs = 362
done = 10
scan vocabs ... 
num vocabs = 450
done = 10
scan vocabs ... 
num vocabs = 435
done = 10
scan vocabs ... 
num vocabs = 598
done = 10
scan vocabs ... 
num vocabs = 550
done = 10
scan vocabs ... 
num vocabs = 649
done = 10
scan vocabs ... 
num vocabs = 683
done = 10
scan